In [25]:
from pyspark.sql import SparkSession
from pyspark.sql import column
import pandas as pd
import sqlite3

In [26]:
con = sqlite3.connect('sentiment.db')

In [27]:
df = pd.read_sql_query("SELECT * FROM sentiment", con)
df.columns

Index(['index', 'screen_name', 'date_time', 'text', 'retweeted', 'lang'], dtype='object')

In [28]:
print(df.head)

<bound method NDFrame.head of     index      screen_name                   date_time  \
0       0      ethayes2010  2021-09-04 12:48:33.000000   
1       1     ihave5broats  2021-09-04 12:48:31.000000   
2       2   ToasterJessica  2021-09-04 12:48:31.000000   
3       3      Zed18978371  2021-09-04 12:48:31.000000   
4       4      GunPangSSam  2021-09-04 12:48:30.000000   
5       0          Kahv6m7  2021-09-04 13:28:17.000000   
6       1        TeslaOwls  2021-09-04 13:28:16.000000   
7       2        andyro233  2021-09-04 13:28:15.000000   
8       3       vin_wiesel  2021-09-04 13:28:11.000000   
9       4  Jeeeeee66023780  2021-09-04 13:28:09.000000   
10      0    Coinhustler84  2021-09-04 13:28:57.000000   
11      1   developrwannab  2021-09-04 13:28:53.000000   
12      2     FuBo19562152  2021-09-04 13:28:47.000000   
13      3    kotoMxSamurai  2021-09-04 13:28:47.000000   
14      4      luke_deluxe  2021-09-04 13:28:44.000000   
15      0    Coinhustler84  2021-09-04 13:

In [45]:
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("Tesla Tweets") \
    .getOrCreate()

In [30]:
sparkDF = spark.createDataFrame(df)
sparkDF.printSchema()

root
 |-- index: long (nullable = true)
 |-- screen_name: string (nullable = true)
 |-- date_time: string (nullable = true)
 |-- text: string (nullable = true)
 |-- retweeted: long (nullable = true)
 |-- lang: string (nullable = true)



In [31]:
sparkDF.show()

+-----+---------------+--------------------+-----------------------+---------+----+
|index|    screen_name|           date_time|                   text|retweeted|lang|
+-----+---------------+--------------------+-----------------------+---------+----+
|    0|    ethayes2010|2021-09-04 12:48:...|   RT @ethayes2010: ...|        0|  en|
|    1|   ihave5broats|2021-09-04 12:48:...|   @SamR33v3s New Te...|        0|  ca|
|    2| ToasterJessica|2021-09-04 12:48:...|   RT @basetankrs: T...|        0|  en|
|    3|    Zed18978371|2021-09-04 12:48:...|   @NorristownN @Rol...|        0|  en|
|    4|    GunPangSSam|2021-09-04 12:48:...|@tesla_somac 저도 같...|        0|  ko|
|    0|        Kahv6m7|2021-09-04 13:28:...|   RT @DusundukceVar...|        0|  tr|
|    1|      TeslaOwls|2021-09-04 13:28:...|   @EconomicManBlog ...|        0|  en|
|    2|      andyro233|2021-09-04 13:28:...|   @javielon95 @foro...|        0|  es|
|    3|     vin_wiesel|2021-09-04 13:28:...|   @nuc_geek Media: ...|        0| 

In [32]:
sparkDFnoRT = sparkDF.filter(~sparkDF.text.startswith('RT'))
spark_DF_En = sparkDFnoRT.filter(sparkDFnoRT.lang.contains('en'))
# drop column index
spark_DF_No_index = spark_DF_En.drop('index')

# df.filter(df.name.startswith('Al')).collect()

In [33]:
sparkDFnoRT.show()

+-----+---------------+--------------------+-----------------------+---------+----+
|index|    screen_name|           date_time|                   text|retweeted|lang|
+-----+---------------+--------------------+-----------------------+---------+----+
|    1|   ihave5broats|2021-09-04 12:48:...|   @SamR33v3s New Te...|        0|  ca|
|    3|    Zed18978371|2021-09-04 12:48:...|   @NorristownN @Rol...|        0|  en|
|    4|    GunPangSSam|2021-09-04 12:48:...|@tesla_somac 저도 같...|        0|  ko|
|    1|      TeslaOwls|2021-09-04 13:28:...|   @EconomicManBlog ...|        0|  en|
|    2|      andyro233|2021-09-04 13:28:...|   @javielon95 @foro...|        0|  es|
|    3|     vin_wiesel|2021-09-04 13:28:...|   @nuc_geek Media: ...|        0|  en|
|    4|Jeeeeee66023780|2021-09-04 13:28:...|   @TeslaAIBot @Tesl...|        0|  en|
|    1| developrwannab|2021-09-04 13:28:...|   @ukspacebulletin ...|        0|  en|
|    4|    luke_deluxe|2021-09-04 13:28:...|   @MMarboni @RARpar...|        0| 

In [34]:
spark_DF_En.show()

+-----+---------------+--------------------+--------------------+---------+----+
|index|    screen_name|           date_time|                text|retweeted|lang|
+-----+---------------+--------------------+--------------------+---------+----+
|    3|    Zed18978371|2021-09-04 12:48:...|@NorristownN @Rol...|        0|  en|
|    1|      TeslaOwls|2021-09-04 13:28:...|@EconomicManBlog ...|        0|  en|
|    3|     vin_wiesel|2021-09-04 13:28:...|@nuc_geek Media: ...|        0|  en|
|    4|Jeeeeee66023780|2021-09-04 13:28:...|@TeslaAIBot @Tesl...|        0|  en|
|    1| developrwannab|2021-09-04 13:28:...|@ukspacebulletin ...|        0|  en|
|    4|    luke_deluxe|2021-09-04 13:28:...|@MMarboni @RARpar...|        0|  en|
|    1| developrwannab|2021-09-04 13:28:...|@ukspacebulletin ...|        0|  en|
|    4|    luke_deluxe|2021-09-04 13:28:...|@MMarboni @RARpar...|        0|  en|
|    0|  lordhyperkill|2021-09-04 14:52:...|@tylertootatted @...|        0|  en|
|    1|     barryagall|2021-

In [38]:
# need to fix index column

spark_DF_No_index.show()

+---------------+--------------------+--------------------+---------+----+
|    screen_name|           date_time|                text|retweeted|lang|
+---------------+--------------------+--------------------+---------+----+
|    Zed18978371|2021-09-04 12:48:...|@NorristownN @Rol...|        0|  en|
|      TeslaOwls|2021-09-04 13:28:...|@EconomicManBlog ...|        0|  en|
|     vin_wiesel|2021-09-04 13:28:...|@nuc_geek Media: ...|        0|  en|
|Jeeeeee66023780|2021-09-04 13:28:...|@TeslaAIBot @Tesl...|        0|  en|
| developrwannab|2021-09-04 13:28:...|@ukspacebulletin ...|        0|  en|
|    luke_deluxe|2021-09-04 13:28:...|@MMarboni @RARpar...|        0|  en|
| developrwannab|2021-09-04 13:28:...|@ukspacebulletin ...|        0|  en|
|    luke_deluxe|2021-09-04 13:28:...|@MMarboni @RARpar...|        0|  en|
|  lordhyperkill|2021-09-04 14:52:...|@tylertootatted @...|        0|  en|
|     barryagall|2021-09-04 14:52:...|@jrtolver16 @Tesl...|        0|  en|
+---------------+--------

In [41]:
pandasDF = spark_DF_No_index.toPandas()

In [42]:
pandasDF

,screen_name,date_time,text,retweeted,lang
0,Zed18978371,2021-09-04 12:48:31.000000,@NorristownN @RolandStautz Yup - I’m the crazy...,0,en
1,TeslaOwls,2021-09-04 13:28:16.000000,@EconomicManBlog @pauluaq1 @jrtolver16 @Tesla ...,0,en
2,vin_wiesel,2021-09-04 13:28:11.000000,@nuc_geek Media: “TESLA CRASHES INTO BUILDING ...,0,en
3,Jeeeeee66023780,2021-09-04 13:28:09.000000,@TeslaAIBot @Tesla Why you have so many follow...,0,en
4,developrwannab,2021-09-04 13:28:53.000000,@ukspacebulletin Great thanks. Now I got to ad...,0,en
5,luke_deluxe,2021-09-04 13:28:44.000000,@MMarboni @RARpartners @TechInsider @OOIDA Fin...,0,en
6,developrwannab,2021-09-04 13:28:53.000000,@ukspacebulletin Great thanks. Now I got to ad...,0,en
7,luke_deluxe,2021-09-04 13:28:44.000000,@MMarboni @RARpartners @TechInsider @OOIDA Fin...,0,en
8,lordhyperkill,2021-09-04 14:52:30.000000,@tylertootatted @TheSmokingTire That’s a real ...,0,en
9,barryagall,2021-09-04 14:52:29.000000,@jrtolver16 @Tesla White interior.,0,en


In [43]:
from sqlite3 import connect
from sqlalchemy import create_engine

In [44]:
engine = create_engine('sqlite:///sentiment.db', echo = True)
sqlite_connection = engine.connect()
sqlite_table = "spark_cleaned"
pandasDF.to_sql(sqlite_table, sqlite_connection, if_exists = 'replace')
sqlite_connection.close()

2021-09-05 13:40:12,948 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-09-05 13:40:12,952 INFO sqlalchemy.engine.base.Engine ()
2021-09-05 13:40:12,955 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-09-05 13:40:12,956 INFO sqlalchemy.engine.base.Engine ()
2021-09-05 13:40:12,969 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("spark_cleaned")
2021-09-05 13:40:12,974 INFO sqlalchemy.engine.base.Engine ()
2021-09-05 13:40:12,993 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("spark_cleaned")
2021-09-05 13:40:13,012 INFO sqlalchemy.engine.base.Engine ()
2021-09-05 13:40:13,015 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-09-05 13:40:13,019 INFO sqlalchemy.engine.base.Engine ()
2021-09-05 13:40:13,027 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_xinfo("spark_cleaned")
2021-09-05 13:40:13,029 INFO sq